In [ ]:
#
#/*
# APROPOS - Organization for the EU Horizon 2020 APROPOS project, grant no. 956090.
# Copyright 2023 Antoine Grenier, Jie Lei, Hans Jakob Damsgaard
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# */
#

 # _     _ _______  ______ ______       _______ __   __ ______   ______
 # |_____| |_____| |_____/ |     \      |______   \_/   |     \ |_____/
 # |     | |     | |    \_ |_____/      ______|    |    |_____/ |    \_
                                                                     
# This notebook will demonstrate using FPGA to accelerate GNSS algorithm. 
# The is work is within the scope of Hard SyDR project.
# Visit the Repo at: REPO_URL_HERE

In [ ]:
# To program the FPGA and show the info regarding the IP and the name of top function
# The top_hls_hard_sydr_0 is the HLS IP that we generated from HLS flow.
from pynq import Overlay
# PYNQ version 3.0.1, as of this tutorial
ol = Overlay("./hard_sydr_FPGA.bit")
ol.ip_dict 

In [ ]:
# Create aliases for calling of the overlay and send and receive channel of the DMA block
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

hls_ip = ol.top_hls_hard_sydr_0

In [ ]:
# For debugging purpose, user can inspect the register map of the IP to see if the IP is running, stalled or idle
hls_ip.register_map

In [ ]:
# To start the IP, we can write the bit into the 0x81 address, 
# The writing only need to be done once
CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
hls_ip.register_map

In [ ]:
from pynq import allocate
import numpy as np
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [ ]:
# Define a function that transfer and receive data to the IP via the DMA
# Wait DMA to finish its transmission, it identifies stalling of the stream which is a common design error by users
def runKernel(inputdata, output_buffer):
    dma_send.transfer(inputdata)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()

In [ ]:
#
# This is a full demo that read the raw input signal from bin file then process them, 
# For a smaller demo, de runHSYDR_demo
#
def runHSYDR_full(MINISECOND, SAVE_OUT_2FILE):
    oneMiniSec_int64Data = int (20000 * 2 / 8)
    rfdata_int64 = allocate(shape=(oneMiniSec_int64Data,), dtype=np.int64)
    output_buffer = allocate(shape=(6,), dtype=np.int64)

    # Path to the binary file
    filepath = "../../Novatel_20211130_resampled_10MHz_8bit_IQ_gain25.bin"
    output_filename = "track_PL.txt"

    miniSecond = MINISECOND

    # run kernel onece, we skipped the acq,
    runKernel(rfdata_int64, output_buffer) 

    # Open the file for reading in binary mode
    # with open(filepath, 'rb') as file:
    with open(filepath, 'rb') as file, open(output_filename, 'w') as ofs:
        # ofs.write("Current Time and Date: " + formatted_time + '\n')
        if SAVE_OUT_2FILE == 0:
            ofs.write("You have chosen not to save run results into file\n")
        for i_minSec in  tqdm(range(miniSecond)):
            for cnt in range(oneMiniSec_int64Data):
                # Read 8 bytes from the file
                bytes_data = file.read(8)
                # Check if we have reached the end of the file
                if not bytes_data:
                    break
                # Convert bytes to a 64-bit signed integer and store it in the array
                rfdata_int64[cnt] = int.from_bytes(bytes_data, byteorder='big', signed=True)

            # roll back the position when reading the file
            # in the c sydr, index jump every 2500 data point, 
            file.seek(-20000, 1) 

            # Check if we reached the end of the file inside the inner loop
            if not bytes_data:
                print("End of file reached.")
                break

            runKernel(rfdata_int64, output_buffer) 
            print 
            if SAVE_OUT_2FILE:
                # Write output_buffer values to the file
                ofs.write(' '.join(map(str, output_buffer)) + '\n')

                
    print("Last tracking output: ", output_buffer)
    print("Done all, check track_PL.txt")
    print("Called runKernel() " + str(miniSecond) + " times\n")
    print( "██████    ██████    ███    ███   ██████   ██       ███████   ████████  ███████ ")
    print( "██       ██    ██   ████  ████   ██   ██  ██       ██           ██     ██      ")
    print( "██       ██    ██   ██ ████ ██   ██████   ██       █████        ██     █████   ")
    print( "██       ██    ██   ██  ██  ██   ██       ██       ██           ██     ██      ")
    print( " ██████   ██████    ██      ██   ██       ███████  ███████      ██     ███████")
    del rfdata_int64, output_buffer


In [ ]:

#
# Note, this is a small demo that read 300 ms of data from a rfdata_int64_300ms.txt file
# the reference output should be track_PL_300ms_ref_results.txt
#
def runHSYDR_demo(MINISECOND, SAVE_OUT_2FILE):
    oneMiniSec_int64Data = int (20000 * 2 / 8)
    rfdata_int64 = allocate(shape=(oneMiniSec_int64Data,), dtype=np.int64)
    output_buffer = allocate(shape=(6,), dtype=np.int64)

    # Path to the binary file
    output_filename = "track_PL.txt"
    rfdata_filepath = "rfdata_int64_300ms.txt"
    
    miniSecond = MINISECOND
    runKernel(rfdata_int64, output_buffer) 

    data = []
    with open(rfdata_filepath, 'r') as file:
        for line in file:
            # Convert line to a number, e.g., int or float
            number = int(line.strip())  # or float(line.strip())
            data.append(number)
    np_array = np.array(data)

    with open(output_filename, 'w') as ofs:
        for i in tqdm(range(miniSecond)):
            for ii in range(oneMiniSec_int64Data):
                rfdata_int64[ii] = np_array[ii+i*oneMiniSec_int64Data]
            runKernel(rfdata_int64, output_buffer) 

            if SAVE_OUT_2FILE:
                # Write output_buffer values to the file
                ofs.write(' '.join(map(str, output_buffer)) + '\n')

                            
    print("Last tracking output: ", output_buffer)
    print("Done all, check track_PL.txt")
    print("Called runKernel() " + str(miniSecond) + " times\n")
    print( "██████    ██████    ███    ███   ██████   ██       ███████   ████████  ███████ ")
    print( "██       ██    ██   ████  ████   ██   ██  ██       ██           ██     ██      ")
    print( "██       ██    ██   ██ ████ ██   ██████   ██       █████        ██     █████   ")
    print( "██       ██    ██   ██  ██  ██   ██       ██       ██           ██     ██      ")
    print( " ██████   ██████    ██      ██   ██       ███████  ███████      ██     ███████")
    del rfdata_int64, output_buffer


In [ ]:
# User can define how long of input signal should be processed
MINISECOND= 300
SAVE_OUT_2FILE = 1
# runHSYDR_full(MINISECOND, SAVE_OUT_2FILE)
runHSYDR_demo(MINISECOND, SAVE_OUT_2FILE)
